In [1]:
""" Filter based on h3 hexagons mapped to polygons

aws emr add-steps --cluster-id <Your EMR cluster id> --steps Type=spark,Name=TestJob,Args=[--deploy-mode,cluster,--master,yarn,--conf,spark.yarn.submit.waitAppCompletion=true,s3a://your-source-bucket/code/pythonjob.py,s3a://your-source-bucket/data/data.csv,s3a://your-destination-bucket/test-output/],ActionOnFailure=CONTINUE
"""

from collections import namedtuple
import logging
import sys


import pyproj

from geopy.distance import great_circle
import pandas as pd
import geopandas as gpd
import numpy as np
import os

from datetime import timedelta, date, datetime
from statistics import *

from pyspark import SparkContext

from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType,
    LongType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
    TimestampType,
    ArrayType
)
from pyspark.sql.functions import (
    from_utc_timestamp,
    to_utc_timestamp,
    dayofyear,
    col,
    unix_timestamp,
    monotonically_increasing_id,
    pandas_udf,
    PandasUDFType,
    col,
    asc,
    lit,
    countDistinct,
)
from sedona.register import SedonaRegistrator  
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import udf
from sedona.utils.adapter import Adapter
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.SpatialRDD import PointRDD, SpatialRDD, CircleRDD
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType, IndexType
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import JoinQuery
from sedona.core.enums import IndexType
from sedona.core.formatMapper.disc_utils import load_spatial_rdd_from_disc, GeoType
from sedona.core.formatMapper import WktReader, GeoJsonReader
import pyspark.sql.functions as F
from math import *
import time

from shapely.wkt import loads as wkt_loads
from shapely.geometry import Point, Polygon, shape
from shapely.ops import transform
import shapely

def create_rectangular_grid(xmin, ymin, xmax, ymax, width, height, crs="epsg:4326"):
    rows = int(np.ceil((ymax-ymin) / height))
    cols = int(np.ceil((xmax-xmin) / width))
    polygons = []
    
    for i in range(rows):
        for j in range(cols):
            min_x = xmin + j * width
            min_y = ymin + i * height
            max_x = min_x + width
            max_y = min_y + height
            polygon = Polygon([(min_x, min_y), (min_x, max_y), (max_x, max_y), (max_x, min_y), (min_x, min_y)])
            polygons.append(polygon)
            
    return gpd.GeoDataFrame(pd.DataFrame({'geometry': polygons}), crs=crs)



spark = (SparkSession.builder.appName("sedona")
                 .config("spark.serializer", KryoSerializer.getName)          
        .config("spark.kryo.registrator",     
                  SedonaKryoRegistrator.getName)    
         .config("spark.driver.maxResultSize", "3g")
    .getOrCreate() 
        )

# Register Sedona UDTs and UDFs
SedonaRegistrator.registerAll(spark)

#------- 
# parameters
#-------

data_ukr = "s3://ipsos-dvd/ukr/data/"
dir_data = "s3://ipsos-dvd/fdd/data/ukr/"



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1697053424101_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
# # create grid to overlay
# crs = "epsg:6381"
# ukr = gpd.read_file(os.path.join(dir_data, 'raw', 'geo', 'ukr_admin_level1'))
# test = ukr.to_crs(crs).dissolve().bounds
# xmin, ymin, xmax, ymax = tuple(test.loc[0])
# polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])
# grid_width = 1000
# grid_height = 1000

# grid = create_rectangular_grid(xmin, ymin, xmax, ymax, grid_width, grid_height, crs)
# polygon_gdf = gpd.GeoDataFrame(pd.DataFrame({'geometry': [polygon]}), crs=crs)
# overlay = gpd.overlay(polygon_gdf, grid, how='intersection')

# overlay.to_file(os.path.join(dir_data, 'out', 'geo',  'overlay.geojson'), driver="GeoJSON")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
fn = os.path.join(dir_data, 'out', 'geo',  'overlay.geojson')
test = gpd.read_file(fn)

poly_spark = spark.createDataFrame(
  test
).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
poly_rdd = Adapter.toSpatialRdd(poly_spark, "geometry")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# # convert pings to sedona geometry format

# path = "s3://ipsos-dvd/ukr/data/pings_2022-01-01_2022-12-31/"

# essential_fields = [
#         StructField("utc_timestamp",LongType(),False),
#         StructField("caid",StringType(),False),
#         StructField("latitude",DoubleType(),False),
#         StructField("longitude",DoubleType(),False),
#         StructField("altitude",DoubleType(),False),
# ]
# raw_schema = StructType(
#     essential_fields + [
#         StructField("id_type",StringType(),False),
#         StructField("geo_hash",StringType(),False),
#         StructField("horizontal_accuracy",DoubleType(),False),
#         StructField("ip_address",StringType(),False),
#         #StructField("altitude",DoubleType(),False),
#         StructField("iso_country_code",StringType(),False)]
# )

# pings = spark.read.schema(raw_schema).parquet(path).select("latitude", "longitude", "caid", "utc_timestamp") # .show() # .limit(1000)
# pings.createOrReplaceTempView("pings")

# # Read Hive table
# pings = spark.sql(
#       """SELECT ST_FlipCoordinates(ST_Transform(ST_FlipCoordinates(ST_Point(cast(pings.longitude as Decimal(24,20)), cast(pings.latitude as Decimal(24,20)))), "epsg:4326", "epsg:6381")) AS point, 
#       utc_timestamp, caid
#       FROM pings;
#       """
# )
# pings.write.format("geoparquet").mode("overwrite").parquet(dir_data + "pings_sedona.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:

num_partitions = 1000

pings = spark.read.parquet(dir_data + "pings_sedona.parquet") # .sample(fraction=0.1)

pings = pings.withColumn("date", F.to_date(F.from_utc_timestamp(F.col("utc_timestamp").cast(TimestampType()), "EET")))
# Repartition the data using the salted repartition column
pings = pings.repartition(num_partitions)
pings = pings.cache()
 
 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# pings = pings.alias("pings").join(F.broadcast(poly_spark).alias("poly_rdd"), F.expr(
#   f"""ST_Within(pings.point, poly_rdd.geometry)"""
# )) #select("pings.utc_timestamp", "pings.caid").show()

# pings.write.mode("overwrite").parquet(os.path.join(dir_data, 'pings_grid'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
grid_type = GridType.QUADTREE # this shit works so much better for skewed data

points_rdd = Adapter.toSpatialRdd(pings, "point")
points_rdd.analyze()
points_rdd.spatialPartitioning(grid_type)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [14]:
#poly_rdd = Adapter.toSpatialRdd(polygons_rdd, "geometry")
poly_rdd.analyze()
poly_rdd.spatialPartitioning(points_rdd.getPartitioner())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:

build_on_spatial_partitioned_rdd = True ## Set to TRUE only if run join query
using_index = True
points_rdd.buildIndex(IndexType.QUADTREE, build_on_spatial_partitioned_rdd)

# points_rdd = points_rdd.cache()
# poly_rdd = poly_rdd.cache()

# #poly_rdd = Adapter.toSpatialRdd(polygons_rdd, "geometry")
# poly_rdd.analyze()
# # poly_rdd.buildIndex(grid_type, build_on_spatial_partitioned_rdd)
# poly_rdd.spatialPartitioning(points_rdd.getPartitioner())

# spatial join
result = JoinQueryRaw.SpatialJoinQueryFlat(points_rdd, poly_rdd, using_index, True)



# (Adapter.toDf(result, poly_rdd.fieldNames, points_rdd.fieldNames, spark)#.show(5)
#     .write.mode("overwrite").parquet(os.path.join(dir_data, 'pings_grid'))
# ) # this is currently slow, may need to remove unrealistically large polygons?


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
(Adapter.toDf(result, poly_rdd.fieldNames, points_rdd.fieldNames, spark)#.show(5)
    .write.mode("overwrite").parquet(os.path.join(dir_data, 'pings_grid'))
) # this is currently slow, may need to remove unrealistically large polygons?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## After Spatial Join

In [2]:
pings_grid = spark.read.parquet(os.path.join(dir_data, 'pings_grid'))


pings_grid = pings_grid.withColumnRenamed("leftgeometry", "geometry")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
pings_grid = pings_grid.groupby("date", "geometry").agg(F.count("caid").alias("count"), F.countDistinct("caid").alias("caid"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
fn = os.path.join(dir_data, 'out', 'geo',  'overlay.geojson')
test = gpd.read_file(fn)

poly_spark = spark.createDataFrame(
  test
).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
dates = pings_grid.select("date").distinct().toPandas()

In [ ]:
# replicate grid data for each date in pings_grid

dates_df = spark.createDataFrame(dates)




In [ ]:
balanced = dates_df.crossJoin(poly_spark)


In [ ]:
# balance panel

pings_grid_full = balanced.join(pings_grid, on = ['date', 'geometry'], how="left")


In [9]:
pings_grid = pings_grid.cache()
pings_grid_full.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

431526000

In [11]:
pings_grid_full.write.parquet(os.path.join(dir_data, "pings_grid_full"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Restart with balanced panel

In [16]:
pings_grid_full = spark.read.parquet(os.path.join(dir_data, "pings_grid_full"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# calculate average in january
pings_grid_full = pings_grid_full.cache()
pings_grid_full.select(F.month("date")).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|month(date)|
+-----------+
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
|         12|
+-----------+
only showing top 20 rows

In [18]:
pings_grid_full = pings_grid_full.na.fill({'count': 0, 'caid': 0})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# calculate the conditional average
temp = pings_grid_full.filter(F.month("date") == 1).groupby("geometry").agg((F.sum(F.col("count") * (F.col("count") > 0).cast(IntegerType())) / F.sum((F.col("count") > 0).cast(IntegerType()))).alias("count_jan"), 
                                                            (F.sum(F.col("caid") * (F.col("caid") > 0).cast(IntegerType())) / F.sum((F.col("caid") > 0).cast(IntegerType()))).alias("caid_jan"))
pings_grid_full = pings_grid_full.join(temp, on = "geometry", how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
pings_grid_full = pings_grid_full.na.fill({'count_jan' : 0, 'caid_jan' : 0})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
pings_grid_full = (pings_grid_full.withColumn("count_ratio", F.when(F.col("count_jan") == 0, 0).otherwise(F.col("count") / F.col("count_jan")))
                                .withColumn("caid_ratio", F.when(F.col("caid_jan") == 0, 0).otherwise(F.col("caid") / F.col("caid_jan")))
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
dates = pings_grid_full.select("date").distinct().toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
pings_grid_full = pings_grid_full.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
import boto3
import re
s3 = boto3.client('s3')

def get_date_parts_from_s3_folder(bucket_name, prefix):
    """Get date parts from .geojson files in the specified S3 folder."""
    date_parts = []

    # List the files in the given S3 folder
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Check if there are any files in the folder
    if response.get('Contents'):
        for obj in response['Contents']:
            file_name = obj['Key'].split('/')[-1]  # Extract the file name from the full S3 key

            # Use a regular expression to match the date part of the filename
            match = re.match(r'(\d{4}-\d{2}-\d{2})\.geojson', file_name)
            if match:
                date_parts.append(match.group(1))

    return date_parts

skip = get_date_parts_from_s3_folder("ipsos-dvd", "fdd/data/out/ukr/grid_daily_pings")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
for date in ['2022-11-09']: # dates['date']:
    
    print(date)
    
    gdf_date = pings_grid_full.filter(F.col("date") == date).toPandas()
    gdf_date = gpd.GeoDataFrame(gdf_date, geometry = gdf_date['geometry'])
    gdf_date.to_file("s3://ipsos-dvd/fdd/data/out/ukr/" + "grid_daily_pings/" + date + ".geojson", driver="GeoJSON")
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2022-11-09

In [ ]:
date



In [ ]:
pings_gdf['day'] = pings_gdf['day'].astype(str)
pings_gdf[['day', 'geometry', 'count', 'caid']].to_file(os.path.join(dir_data, "grid_daily_pings_raw.geojson"), driver = "GeoJSON")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pings_gdf = gpd.read_file(os.path.join(dir_data, "grid_daily_pings_raw.geojson"))

In [5]:
pings_gdf['date'] = pd.to_datetime(pings_gdf['date'])
pings_gdf['day'] = pings_gdf['date'].dt.date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# balance panel by adding missing weeks
# Set "week" and "index_right" as a MultiIndex
test = pings_gdf.copy()
test['geom_id'] = test['geometry'].to_wkt.apply(hash)
test = test.set_index(['day', 'geom_id'])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
unhashable type: 'Polygon'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/geopandas/geoseries.py", line 624, in apply
    result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/series.py", line 4357, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/apply.py", line 1043, in apply
    return self.apply_standard()
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/apply.py", line 1101, in apply_standard
    convert=self.convert_dtype,
  File "pandas/_libs/lib.pyx", line 2859, in pandas._libs.lib.map_infer
TypeError: unhashable type: 'Polygon'



In [ ]:

# # Create all possible combinations of "week" and "index_right"
# unique_weeks = test.index.levels[0]
# unique_index_right = test.index.levels[1]
# index_combinations = pd.MultiIndex.from_product([unique_weeks, unique_index_right], names=['day', 'geometry'])

# # Reindex the DataFrame with the new index combinations
# test = test.reindex(index_combinations).reset_index()

In [49]:
unique_index_right

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'unique_index_right' is not defined
Traceback (most recent call last):
NameError: name 'unique_index_right' is not defined



In [41]:
pings_grid['geometry'].astype(str).nunique()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

227265